In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
data_path = "C:\Bhavesh\Python\python_work\BrainTumorClassification\Data\Training"

batch_size = 10
img_height = 512
img_width = 512

In [ ]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
  data_path,
  validation_split=0.3,
  subset="training",
  seed=0,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_path,
    validation_split=0.3,
    subset="validation",
    seed=0,
    image_size=(img_height, img_width),
    batch_size=batch_size)

In [ ]:
class_names = train_data.class_names
print(class_names)

In [ ]:
num_classes = 4

model = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
])

num_classes = 4

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
epochs = 3

model.fit(
  train_data,
  validation_data=val_data,
  epochs=epochs)

In [ ]:
test_path = "C:\Bhavesh\Python\python_work\BrainTumorClassification\Data\Testing"

test_data = tf.keras.preprocessing.image_dataset_from_directory(
  test_path,
  validation_split=0.5,
  subset="training",
  seed=1,
  image_size=(img_height, img_width),
  batch_size=batch_size)

eval = model.evaluate(test_data)
print("test loss, test acc:", eval)

In [ ]:
num_classes = 4

model2 = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(.1),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)

model2.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
epochs = 2

Callbacks = [tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)]

model2.fit(
  train_data,
  validation_data=val_data,
  epochs=epochs)

In [ ]:
test_path = "/content/drive/MyDrive/Data/Data/Testing"

test_data = tf.keras.preprocessing.image_dataset_from_directory(
  test_path,
  validation_split=0.01,
  subset="training",
  seed=1,
  image_size=(img_height, img_width),
  batch_size=batch_size)

eval = model2.evaluate(test_data)
print("test loss, test acc:", eval)

In [ ]:
!pip install scikit-optimize

In [ ]:
import skopt
from skopt import gp_minimize, forest_minimize
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_convergence
from skopt.plots import plot_objective, plot_evaluations
from skopt.plots import plot_histogram, plot_objective_2D
from skopt.utils import use_named_args

In [ ]:
dim_learning_rate = Real(low=1e-6, high=1e-2, prior='log-uniform', name='learning_rate')
dim_num_dense_layers = Integer(low=1, high=5, name='num_dense_layers')
dim_num_dense_nodes = Integer(low=5, high=512, name='num_dense_nodes')
dim_activation = Categorical(categories=['relu'], name='activation')

dimensions = [dim_learning_rate, dim_num_dense_layers, dim_num_dense_nodes, dim_activation]

In [ ]:
default_parameters = [5.8e-04, 1, 269, 'relu']
num_classes = 4

In [ ]:
def create_model(learning_rate, num_dense_layers, num_dense_nodes, activation):
  model = tf.keras.Sequential()

  model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1./255))

  model.add(tf.keras.layers.Conv2D(32, 3, activation=activation))
  model.add(tf.keras.layers.MaxPooling2D())

  model.add(tf.keras.layers.Conv2D(32, 3, activation=activation))
  model.add(tf.keras.layers.MaxPooling2D())

  model.add(tf.keras.layers.Conv2D(32, 3, activation=activation))
  model.add(tf.keras.layers.MaxPooling2D())

  model.add(tf.keras.layers.Flatten())

  for i in range(num_dense_layers):
    name = f'dense_layer_{i + 1}'
    model.add(tf.keras.layers.Dense(num_dense_nodes, activation=activation, name=name))

  model.add(tf.keras.layers.Dense(num_classes))

  optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

  model.compile(optimizer=optimizer, loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

  return model

In [ ]:
best_accuracy = 0.0

@use_named_args(dimensions=dimensions)
def fitness(learning_rate, num_dense_layers, num_dense_nodes, activation):
    print('learning rate: {0:.1e}'.format(learning_rate))
    print('num_dense_layers:', num_dense_layers)
    print('num_dense_nodes:', num_dense_nodes)
    print('activation:', activation)
    print()

    model = create_model(learning_rate=learning_rate, num_dense_layers=num_dense_layers, num_dense_nodes=num_dense_nodes, activation=activation)

    history = model.fit(train_data, validation_data=val_data, epochs=5, batch_size=128)

    accuracy = history.history['val_accuracy'][-1]
    print("Accuracy: {0:.2%}".format(accuracy))
    print()

    global best_accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
    
    del model

    return -accuracy

In [ ]:
fitness(default_parameters)

In [ ]:
search_result = gp_minimize(func=fitness,
                            dimensions=dimensions,
                            acq_func='EI', # Expected Improvement.
                            n_calls=40,
                            x0=default_parameters)

In [ ]:
best = search_result.x
print(f"Best hyper-parameters: {best}")

best_acc = round(abs(search_result.fun), 2)
print(f"Best accuracy: {best_acc}")


'''
Best:
    learning rate: 5.8e-04
    num_dense_layers: 1
    num_dense_nodes: 269
    activation: relu
    epochs: 5
'''